In [1]:
import pandas as pd 
from fbprophet import Prophet
import itertools
import numpy as np
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
import datetime



Importing plotly failed. Interactive plots will not work.


In [21]:
occupancy = pd.read_csv('../data/jghOccupancy.csv', parse_dates=['ds'])
occupancy.head()

,ds,y
0,2012-05-01 00:00:00,39.0
1,2012-05-01 01:00:00,40.0
2,2012-05-01 02:00:00,40.0
3,2012-05-01 03:00:00,40.0
4,2012-05-01 04:00:00,39.0


In [15]:
weather = pd.read_csv('../data/CYULweather.csv', parse_dates=['valid'])
weather.head()

,station,valid,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,...,skyl4,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar
0,CYUL,2021-01-01 00:00:00,21.2,15.8,79.31,270.00,5.0,0.0,30.42,1030.6,...,M,M,M,M,M,M,M,M,13.57,CYUL 010500Z 27005KT 15SM BKN035 M06/M09 A3042...
1,CYUL,2021-01-01 01:00:00,23.0,17.6,79.47,280.00,2.0,0.0,30.45,1031.6,...,M,M,M,M,M,M,M,M,M,CYUL 010600Z 28002KT 15SM OVC033 M05/M08 A3045...
2,CYUL,2021-01-01 02:00:00,23.0,17.6,79.47,M,2.0,0.0,30.46,1031.8,...,M,M,M,M,M,M,M,M,M,CYUL 010700Z VRB02KT 15SM OVC032 M05/M08 A3046...
3,CYUL,2021-01-01 03:00:00,24.8,19.4,79.62,250.00,3.0,0.0,30.49,1032.8,...,M,M,M,M,M,M,M,M,20.46,CYUL 010800Z 25003KT 15SM OVC031 M04/M07 A3049...
4,CYUL,2021-01-01 04:00:00,24.8,17.6,73.70,320.00,2.0,0.0,30.49,1032.9,...,M,M,M,M,M,M,M,M,M,CYUL 010900Z 32002KT 15SM OVC031 M04/M08 A3049...


In [16]:
weather_archive = pd.read_csv('../data/CYULweather_archive.csv', parse_dates=['valid'])
weather_archive.head()

/Users/devinhopkins/opt/anaconda3/envs/prophet/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3,4,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,station,valid,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,...,skyl4,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar
0,CYUL,2012-05-01 00:00:00,51.80,28.40,40.25,120.00,3.0,0.0,30.09,1018.90,...,M,M,M,M,M,M,M,M,51.80,CYUL 010400Z 12003KT 15SM OVC085 11/M02 A3009 ...
1,CYUL,2012-05-01 01:00:00,51.80,28.40,40.25,100.00,6.0,0.0,30.05,1017.60,...,M,M,M,M,M,M,M,M,51.80,CYUL 010500Z 10006KT 15SM OVC080 11/M02 A3005 ...
2,CYUL,2012-05-01 01:25:00,51.80,M,M,130.00,8.0,0.0,M,M,...,M,-SHRA,M,M,M,M,M,M,M,CYUL 010525Z 13008KT 15SM -SHRA OVC080 11/ RMK...
3,CYUL,2012-05-01 02:00:00,51.80,30.20,43.32,120.00,10.0,0.0,30.02,1016.60,...,M,-SHRA,M,M,M,M,M,M,51.80,CYUL 010600Z 12010KT 15SM -SHRA BKN080 OVC110 ...
4,CYUL,2012-05-01 03:00:00,50.00,35.60,57.52,160.00,4.0,0.0,30.02,1016.80,...,M,-SHRA,M,M,M,M,M,M,48.44,CYUL 010700Z 16004KT 15SM -SHRA BKN075 OVC100 ...


In [20]:
weather_final = pd.concat([weather_archive, weather], ignore_index=True).rename(columns={'valid':'ds'}).drop('station', axis=1)

In [26]:
df = occupancy.merge(weather_final, on='ds')

In [34]:
columns = df.columns.tolist()
columns

['ds',
 'y',
 'tmpf',
 'dwpf',
 'relh',
 'drct',
 'sknt',
 'p01i',
 'alti',
 'mslp',
 'vsby',
 'gust',
 'skyc1',
 'skyc2',
 'skyc3',
 'skyc4',
 'skyl1',
 'skyl2',
 'skyl3',
 'skyl4',
 'wxcodes',
 'ice_accretion_1hr',
 'ice_accretion_3hr',
 'ice_accretion_6hr',
 'peak_wind_gust',
 'peak_wind_drct',
 'peak_wind_time',
 'feel',
 'metar']

In [43]:
numeric = ['tmpf','dwpf','relh','drct','sknt','p01i','alti','mslp','vsby']
for num in numeric:
    df[num] = pd.to_numeric(df[num], errors='coerce')
    df[num] = df[num].interpolate()
df

,ds,y,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,...,peak_wind_time,feel,metar,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2012-05-01 00:00:00,39.0,51.8,28.4,40.25,120.0,3.0,0.0,30.09,1018.9,...,M,51.80,CYUL 010400Z 12003KT 15SM OVC085 11/M02 A3009 ...,False,True,False,False,False,False,False
1,2012-05-01 01:00:00,40.0,51.8,28.4,40.25,100.0,6.0,0.0,30.05,1017.6,...,M,51.80,CYUL 010500Z 10006KT 15SM OVC080 11/M02 A3005 ...,False,True,False,False,False,False,False
2,2012-05-01 02:00:00,40.0,51.8,30.2,43.32,120.0,10.0,0.0,30.02,1016.6,...,M,51.80,CYUL 010600Z 12010KT 15SM -SHRA BKN080 OVC110 ...,False,True,False,False,False,False,False
3,2012-05-01 03:00:00,40.0,50.0,35.6,57.52,160.0,4.0,0.0,30.02,1016.8,...,M,48.44,CYUL 010700Z 16004KT 15SM -SHRA BKN075 OVC100 ...,False,True,False,False,False,False,False
4,2012-05-01 04:00:00,39.0,48.2,37.4,66.05,130.0,8.0,0.0,30.01,1016.4,...,M,44.07,CYUL 010800Z 13008KT 15SM -SHRA BKN075 OVC090 ...,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77100,2021-03-19 15:00:00,67.0,28.4,3.2,33.41,250.0,10.0,0.0,30.41,1030.0,...,M,18.45,CYUL 191900Z 25010G15KT 190V280 30SM SKC M02/M...,False,False,False,False,True,False,False
77101,2021-03-19 16:00:00,69.0,30.2,12.2,46.62,220.0,12.0,0.0,30.39,1029.3,...,M,19.71,CYUL 192000Z 22012KT 200V280 30SM SKC M01/M11 ...,False,False,False,False,True,False,False
77102,2021-03-19 17:00:00,73.0,30.2,8.6,39.70,240.0,12.0,0.0,30.37,1028.7,...,M,19.71,CYUL 192100Z 24012KT 30SM SKC M01/M13 A3037 RM...,False,False,False,False,True,False,False
77103,2021-03-19 18:00:00,74.0,30.2,10.4,43.04,240.0,14.0,0.0,30.37,1028.6,...,M,18.84,CYUL 192200Z 24014KT 30SM FEW250 M01/M12 A3037...,False,False,False,False,True,False,False


In [37]:
def is_monday(ds):
    date = pd.to_datetime(ds)
    return (date.dayofweek == 0)
def is_tuesday(ds):
    date = pd.to_datetime(ds)
    return (date.dayofweek == 1)
def is_wednesday(ds):
    date = pd.to_datetime(ds)
    return (date.dayofweek == 2)
def is_thursday(ds):
    date = pd.to_datetime(ds)
    return (date.dayofweek == 3)
def is_friday(ds):
    date = pd.to_datetime(ds)
    return (date.dayofweek == 4)
def is_saturday(ds):
    date = pd.to_datetime(ds)
    return (date.dayofweek == 5)
def is_sunday(ds):
    date = pd.to_datetime(ds)
    return (date.dayofweek == 6)



df['monday'] = df['ds'].apply(is_monday)
df['tuesday'] = df['ds'].apply(is_tuesday)
df['wednesday'] = df['ds'].apply(is_wednesday)
df['thursday'] = df['ds'].apply(is_thursday)
df['friday'] = df['ds'].apply(is_friday)
df['saturday'] = df['ds'].apply(is_saturday)
df['sunday'] = df['ds'].apply(is_sunday)

df.tail()

,ds,y,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,...,peak_wind_time,feel,metar,monday,tuesday,wednesday,thursday,friday,saturday,sunday
77100,2021-03-19 15:00:00,67.0,28.4,3.2,33.41,250.0,10.0,0.0,30.41,1030.0,...,M,18.45,CYUL 191900Z 25010G15KT 190V280 30SM SKC M02/M...,False,False,False,False,True,False,False
77101,2021-03-19 16:00:00,69.0,30.2,12.2,46.62,220.0,12.0,0.0,30.39,1029.3,...,M,19.71,CYUL 192000Z 22012KT 200V280 30SM SKC M01/M11 ...,False,False,False,False,True,False,False
77102,2021-03-19 17:00:00,73.0,30.2,8.6,39.70,240.0,12.0,0.0,30.37,1028.7,...,M,19.71,CYUL 192100Z 24012KT 30SM SKC M01/M13 A3037 RM...,False,False,False,False,True,False,False
77103,2021-03-19 18:00:00,74.0,30.2,10.4,43.04,240.0,14.0,0.0,30.37,1028.6,...,M,18.84,CYUL 192200Z 24014KT 30SM FEW250 M01/M12 A3037...,False,False,False,False,True,False,False
77104,2021-03-19 19:00:00,83.0,30.2,8.6,39.70,250.0,14.0,0.0,30.37,1028.9,...,M,18.84,CYUL 192300Z 25014KT 30SM FEW250 M01/M13 A3037...,False,False,False,False,True,False,False


In [39]:
train_years = 1
target_df = df.tail(train_years*365*24)
cross_val_days = 30

param_grid = {  
    'changepoint_prior_scale': [0.001],
    'seasonality_prior_scale': [10],
    'seasonality_mode': ['multiplicative'],
    'changepoint_range': [0.9],
    'daily_seasonality': [True],
    'yearly_seasonality': [False]
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here



# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params)
    m.add_country_holidays(country_name='CA')
    for day in ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']:
        m.add_seasonality(name=day, period=24, fourier_order=1, condition_name=day)
    m.fit(target_df)  # Fit model with given params
    df_cv = cross_validation(m, initial=str(len(target_df)/24-cross_val_days)+' days', period='24 hours', horizon='24 hours', parallel='processes')
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

5


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9be622b640>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  8.933646  


In [42]:
train_years = 1
target_df = df.tail(train_years*365*24)
cross_val_days = 30

param_grid = {  
    'changepoint_prior_scale': [0.001],
    'seasonality_prior_scale': [10],
    'seasonality_mode': ['multiplicative'],
    'changepoint_range': [0.9],
    'daily_seasonality': [True],
    'yearly_seasonality': [False]
}

for regressor in ['tmpf','dwpf','relh']:
    
    print(regressor)
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here



    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params)
        m.add_country_holidays(country_name='CA')
        for day in ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']:
            m.add_seasonality(name=day, period=24, fourier_order=1, condition_name=day)
        m.add_regressor(regressor)
        m.fit(target_df)  # Fit model with given params
        df_cv = cross_validation(m, initial=str(len(target_df)/24-cross_val_days)+' days', period='24 hours', horizon='24 hours', parallel='processes')
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses
    print(tuning_results)

tmpf


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd2939c40>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  8.560517  
dwpf


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bde7d0700>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  9.127494  
relh


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd5f1ba90>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  9.407778  
drct


ValueError: Found NaN in column 'drct'

In [44]:
train_years = 1
target_df = df.tail(train_years*365*24)
cross_val_days = 30

param_grid = {  
    'changepoint_prior_scale': [0.001],
    'seasonality_prior_scale': [10],
    'seasonality_mode': ['multiplicative'],
    'changepoint_range': [0.9],
    'daily_seasonality': [True],
    'yearly_seasonality': [False]
}

for regressor in ['drct','sknt','p01i','alti','mslp','vsby']:
    
    print(regressor)
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here



    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params)
        m.add_country_holidays(country_name='CA')
        for day in ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']:
            m.add_seasonality(name=day, period=24, fourier_order=1, condition_name=day)
        m.add_regressor(regressor)
        m.fit(target_df)  # Fit model with given params
        df_cv = cross_validation(m, initial=str(len(target_df)/24-cross_val_days)+' days', period='24 hours', horizon='24 hours', parallel='processes')
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses
    print(tuning_results)

drct


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd2959280>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  8.967984  
sknt


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd5f09970>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  9.061442  
p01i


...

   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  9.238983  
mslp


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd5f09610>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  9.176588  
vsby


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bed2e3400>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  8.969924  


In [46]:
df.tmpf

0        51.8
1        51.8
2        51.8
3        50.0
4        48.2
         ... 
77100    28.4
77101    30.2
77102    30.2
77103    30.2
77104    30.2
Name: tmpf, Length: 77105, dtype: float64

In [59]:
df['tmpf_12'] = df.tmpf-df.tmpf.rolling(12).mean()
df['tmpf_24'] = df.tmpf-df.tmpf.rolling(24).mean()
df['tmpf_48'] = df.tmpf-df.tmpf.rolling(48).mean()
df['tmpf_72'] = df.tmpf-df.tmpf.rolling(72).mean()
df['tmpf_96'] = df.tmpf-df.tmpf.rolling(96).mean()
df['tmpf_120'] = df.tmpf-df.tmpf.rolling(120).mean()
df['tmpf_144'] = df.tmpf-df.tmpf.rolling(144).mean()
df['tmpf_168'] = df.tmpf-df.tmpf.rolling(168).mean()
df

,ds,y,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,...,saturday,sunday,tmpf_12,tmpf_24,tmpf_48,tmpf_72,tmpf_96,tmpf_120,tmpf_144,tmpf_168
0,2012-05-01 00:00:00,39.0,51.8,28.4,40.25,120.0,3.0,0.0,30.09,1018.9,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-05-01 01:00:00,40.0,51.8,28.4,40.25,100.0,6.0,0.0,30.05,1017.6,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-05-01 02:00:00,40.0,51.8,30.2,43.32,120.0,10.0,0.0,30.02,1016.6,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-05-01 03:00:00,40.0,50.0,35.6,57.52,160.0,4.0,0.0,30.02,1016.8,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-05-01 04:00:00,39.0,48.2,37.4,66.05,130.0,8.0,0.0,30.01,1016.4,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77100,2021-03-19 15:00:00,67.0,28.4,3.2,33.41,250.0,10.0,0.0,30.41,1030.0,...,False,False,8.25,3.075,-3.9375,-2.675,0.97500,3.465,3.5375,2.035714
77101,2021-03-19 16:00:00,69.0,30.2,12.2,46.62,220.0,12.0,0.0,30.39,1029.3,...,False,False,9.00,5.400,-1.9125,-0.925,2.55000,5.220,5.2750,3.889286
77102,2021-03-19 17:00:00,73.0,30.2,8.6,39.70,240.0,12.0,0.0,30.37,1028.7,...,False,False,7.95,5.925,-1.6125,-0.950,2.34375,5.190,5.2125,3.932143
77103,2021-03-19 18:00:00,74.0,30.2,10.4,43.04,240.0,14.0,0.0,30.37,1028.6,...,False,False,6.75,6.450,-1.3125,-0.950,2.15625,5.145,5.1250,3.975000


In [60]:
df.columns.tolist()

['ds',
 'y',
 'tmpf',
 'dwpf',
 'relh',
 'drct',
 'sknt',
 'p01i',
 'alti',
 'mslp',
 'vsby',
 'gust',
 'skyc1',
 'skyc2',
 'skyc3',
 'skyc4',
 'skyl1',
 'skyl2',
 'skyl3',
 'skyl4',
 'wxcodes',
 'ice_accretion_1hr',
 'ice_accretion_3hr',
 'ice_accretion_6hr',
 'peak_wind_gust',
 'peak_wind_drct',
 'peak_wind_time',
 'feel',
 'metar',
 'monday',
 'tuesday',
 'wednesday',
 'thursday',
 'friday',
 'saturday',
 'sunday',
 'tmpf_12',
 'tmpf_24',
 'tmpf_48',
 'tmpf_72',
 'tmpf_96',
 'tmpf_120',
 'tmpf_144',
 'tmpf_168']

In [61]:
train_years = 1
target_df = df.tail(train_years*365*24)
cross_val_days = 30

param_grid = {  
    'changepoint_prior_scale': [0.001],
    'seasonality_prior_scale': [10],
    'seasonality_mode': ['multiplicative'],
    'changepoint_range': [0.9],
    'daily_seasonality': [True],
    'yearly_seasonality': [False]
}

for regressor in ['tmpf', 'tmpf_12',
 'tmpf_24',
 'tmpf_48',
 'tmpf_72',
 'tmpf_96',
 'tmpf_120',
 'tmpf_144',
 'tmpf_168']:
    
    print(regressor)
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here



    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params)
        m.add_country_holidays(country_name='CA')
        for day in ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']:
            m.add_seasonality(name=day, period=24, fourier_order=1, condition_name=day)
        m.add_regressor(regressor)
        m.fit(target_df)  # Fit model with given params
        df_cv = cross_validation(m, initial=str(len(target_df)/24-cross_val_days)+' days', period='24 hours', horizon='24 hours', parallel='processes')
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses
    print(tuning_results)

tmpf


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bde7d04c0>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  8.560517  
tmpf_12


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bed2e3400>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  9.041367  
tmpf_24


...

   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  9.012488  
tmpf_144


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bed2e3400>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  9.025986  
tmpf_168


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd5f1bbe0>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality      rmse  
0                0.9               True               False  8.945771  


In [73]:
ramq_stats = pd.read_csv('../data/ramq_stats.csv', parse_dates=['ds'])
ramq_stats

,holiday,ds
0,victoria day,2012-05-21
1,st-jean,2012-06-25
2,canada day,2012-07-02
3,labour day,2012-09-03
4,thanksgiving,2012-10-08
...,...,...
138,new years eve,2022-12-30
139,new years day,2023-01-02
140,day after new years,2023-01-03
141,good friday,2023-04-07


In [93]:
import datetime
holidays_list = []
ramq_stats = pd.read_csv('../data/ramq_stats.csv', parse_dates=['ds'])
for i, row in ramq_stats.iterrows():
    for hour in range(0,24):
        holidays_list.append({'holiday':row['holiday'],
                              'ds':row['ds']+datetime.timedelta(hours = hour),
                              'lower_window':0, 'upper_window':1})
ramq_holidays = pd.DataFrame(holidays_list)


holidays_list = []
jgh_stats = pd.read_csv('../data/jgh_stats.csv', parse_dates=['ds'])
for i, row in jgh_stats.iterrows():
    for hour in range(0,24):
        holidays_list.append({'holiday':row['holiday'],
                              'ds':row['ds']+datetime.timedelta(hours = hour),
                              'lower_window':0, 'upper_window':1})
jgh_holidays = pd.DataFrame(holidays_list)

print(ramq_holidays.head())
print(jgh_holidays.head())

        holiday                  ds  lower_window  upper_window
0  victoria day 2012-05-21 00:00:00             0             1
1  victoria day 2012-05-21 01:00:00             0             1
2  victoria day 2012-05-21 02:00:00             0             1
3  victoria day 2012-05-21 03:00:00             0             1
4  victoria day 2012-05-21 04:00:00             0             1
        holiday                  ds  lower_window  upper_window
0  victoria day 2012-05-21 00:00:00             0             1
1  victoria day 2012-05-21 01:00:00             0             1
2  victoria day 2012-05-21 02:00:00             0             1
3  victoria day 2012-05-21 03:00:00             0             1
4  victoria day 2012-05-21 04:00:00             0             1


In [68]:
def is_ramq_holiday(ds):
    return (ds.strftime('%Y-%m-%d') in [stat.strftime('%Y-%m-%d') for stat in ramq_stats.ds.tolist()])

df['ramq_holiday'] = df['ds'].apply(is_ramq_holiday)
df.tail(100)

,ds,y,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,...,sunday,tmpf_12,tmpf_24,tmpf_48,tmpf_72,tmpf_96,tmpf_120,tmpf_144,tmpf_168,ramq_holiday
77005,2021-03-15 11:00:00,59.0,8.6,-11.2,39.24,320.0,10.0,0.0,30.48,1032.5,...,False,0.90,-5.700,-10.8750,-15.925,-21.13125,-22.980,-22.9750,-20.989286,False
77006,2021-03-15 12:00:00,65.0,10.4,-5.8,47.15,270.0,10.0,0.0,30.46,1032.0,...,False,2.85,-3.225,-8.8500,-13.750,-19.01250,-21.015,-21.0625,-19.146429,False
77007,2021-03-15 13:00:00,77.0,12.2,-5.8,43.53,280.0,15.0,0.0,30.44,1031.3,...,False,4.65,-0.900,-6.9375,-11.600,-16.91250,-19.065,-19.1625,-17.325000,False
77008,2021-03-15 14:00:00,86.0,15.8,-2.2,44.16,260.0,14.0,0.0,30.42,1030.7,...,False,7.80,3.075,-3.3000,-7.700,-13.06875,-15.375,-15.4875,-13.725000,False
77009,2021-03-15 15:00:00,81.0,17.6,-0.4,44.48,260.0,17.0,0.0,30.40,1029.9,...,False,8.85,5.175,-1.5000,-5.625,-11.02500,-13.515,-13.6250,-11.935714,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77100,2021-03-19 15:00:00,67.0,28.4,3.2,33.41,250.0,10.0,0.0,30.41,1030.0,...,False,8.25,3.075,-3.9375,-2.675,0.97500,3.465,3.5375,2.035714,False
77101,2021-03-19 16:00:00,69.0,30.2,12.2,46.62,220.0,12.0,0.0,30.39,1029.3,...,False,9.00,5.400,-1.9125,-0.925,2.55000,5.220,5.2750,3.889286,False
77102,2021-03-19 17:00:00,73.0,30.2,8.6,39.70,240.0,12.0,0.0,30.37,1028.7,...,False,7.95,5.925,-1.6125,-0.950,2.34375,5.190,5.2125,3.932143,False
77103,2021-03-19 18:00:00,74.0,30.2,10.4,43.04,240.0,14.0,0.0,30.37,1028.6,...,False,6.75,6.450,-1.3125,-0.950,2.15625,5.145,5.1250,3.975000,False


In [87]:
holidays

,holiday,ds,lower_window,upper_window
0,playoff,2008-01-13,0,1
1,playoff,2009-01-03,0,1
2,playoff,2010-01-16,0,1
3,playoff,2010-01-24,0,1
4,playoff,2010-02-07,0,1
5,playoff,2011-01-08,0,1
6,playoff,2013-01-12,0,1
7,playoff,2014-01-12,0,1
8,playoff,2014-01-19,0,1
9,playoff,2014-02-02,0,1


In [94]:
train_years = 1
target_df = df.tail(train_years*365*24)
cross_val_days = 30

param_grid = {  
    'changepoint_prior_scale': [0.001],
    'seasonality_prior_scale': [10],
    'seasonality_mode': ['multiplicative'],
    'changepoint_range': [0.9],
    'daily_seasonality': [True],
    'yearly_seasonality': [False],
    'holidays':[None,ramq_holidays,jgh_holidays]
}


# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here



# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params)
    m.add_country_holidays(country_name='CA')
    for day in ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']:
        m.add_seasonality(name=day, period=24, fourier_order=1, condition_name=day)
    m.add_regressor('tmpf')
    m.fit(target_df)  # Fit model with given params
    df_cv = cross_validation(m, initial=str(len(target_df)/24-cross_val_days)+' days', period='24 hours', horizon='24 hours', parallel='processes')
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bed2e3190>
INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd2882c40>
INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd2961d60>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   
1                    0.001                       10   multiplicative   
2                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality  \
0                0.9               True               False   
1                0.9               True               False   
2                0.9               True               False   

                                            holidays      rmse  
0                                               None  8.560517  
1              holiday                  ds  lower...  8.578221  
2              holiday                  ds  lower...  8.574323  


In [ ]:
train_years = 1
target_df = df.tail(train_years*365*24)
cross_val_days = 30

for upper_window in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]:
    print('upper_window: ', upper_window)
    holidays_list = []
    jgh_stats = pd.read_csv('../data/jgh_stats.csv', parse_dates=['ds'])
    for i, row in jgh_stats.iterrows():
        for hour in range(0,24):
            holidays_list.append({'holiday':row['holiday'],
                                  'ds':row['ds']+datetime.timedelta(hours = hour),
                                  'lower_window':0, 'upper_window':upper_window})
    jgh_holidays = pd.DataFrame(holidays_list)

    param_grid = {  
        'changepoint_prior_scale': [0.001],
        'seasonality_prior_scale': [10],
        'seasonality_mode': ['multiplicative'],
        'changepoint_range': [0.9],
        'daily_seasonality': [True],
        'yearly_seasonality': [False],
        'holidays':[jgh_holidays]
    }


    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here



    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params)
        m.add_country_holidays(country_name='CA')
        for day in ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']:
            m.add_seasonality(name=day, period=24, fourier_order=1, condition_name=day)
        m.add_regressor('tmpf')
        m.fit(target_df)  # Fit model with given params
        df_cv = cross_validation(m, initial=str(len(target_df)/24-cross_val_days)+' days', period='24 hours', horizon='24 hours', parallel='processes')
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses
    print(tuning_results)

upper_window:  1


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd2882d30>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality  \
0                0.9               True               False   

                                            holidays      rmse  
0              holiday                  ds  lower...  8.574323  
upper_window:  2


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd2939220>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality  \
0                0.9               True               False   

                                            holidays      rmse  
0              holiday                  ds  lower...  8.567988  
upper_window:  3


...

INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd5f09c40>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality  \
0                0.9               True               False   

                                            holidays      rmse  
0              holiday                  ds  lower...  8.544661  
upper_window:  6


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd28b4310>


   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                       10   multiplicative   

   changepoint_range  daily_seasonality  yearly_seasonality  \
0                0.9               True               False   

                                            holidays      rmse  
0              holiday                  ds  lower...  8.541712  
upper_window:  7


INFO:fbprophet:Making 44 forecasts with cutoffs between 2021-02-03 19:00:00 and 2021-03-18 19:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f9bd1a45bb0>
